In [ ]:
!pip install nyct-gtfs
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/America/New_York /etc/localtime
!date

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Wed Nov 16 23:39:18 EST 2022


In [26]:


#!/usr/bin/python3
# =============================================================================
#
# Leave Now - Encouraging stress-free travel to your local MTA subway station
# by knowing when it's time to leave home.
#
# Version 2.0 7-Mar-2022 by Rob D <http://github.com/rob718/mta-leavenow>
#
# == Change Log
# v2.0 7-Mar-2022: Rewritten using NYCT-GTFS library by Andrew Dickinson
#
# == Prerequisites
# NYCT-GTFS library https://github.com/Andrew-Dickinson/nyct-gtfs
#
# =============================================================================


from nyct_gtfs import NYCTFeed
import threading
import datetime
import time
#import scrollphathd

# Specifiy your feed access key. See https://api.mta.info/#/AccessKey
my_api_key = 'Qmns82EsNo5ohZTnJFruhaoOCTSC0i9E2He3obav'

# Which subway feed should be used for your stop?
# List of feeds: https://api.mta.info/#/subwayRealTimeFeeds


#my_feed_url = 'https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-nqrw' # Q train, etc. (TW)
# my_feed_url = 'https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs' # 1,2,3,4,5,6,7 train, etc. (TW)

# What station do you want to monitor? 
# http://web.mta.info/developers/data/nyct/subway/Stations.csv
#
# Determine "GTFS Stop ID" and suffix direction.
# For example 'R31N' for "Northbound Atlantic Av - Barclays Ctr"

#print("Train stop is 42 st, Q, etc.")


MAX_WAIT_TO_PRINT = 45

#my_stop_id = 'R16N' # 42nd street Q etc

## WAY HOME
stops_to_work = ['117S',# 116 street, S
                 '120S', # 96, 123 S  (maybe)
                 'R16N', #42nd street 123 North (maybe)
                 
                 ]

feeds_to_work = [
                 'https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs',
                 'https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs',
                 'https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-nqrw',
                 ]

train_names_to_work = ['1',
                       '2 3',
                       'Q',
                       ]

stop_names_long = ['116 street, 1 South',
                   '96 Street, 2 or 3 South',
                   '42nd street, Q North',
                   ]

travel_time_to_next = [4,
                       8,
                       8
                       ]

#### WAY TO WORK:

"""

stops_to_work = ['Q03S',# 72nd street Q etc, S
                 '127N', #42nd street 123 North (maybe)
                 '120N', # 96 123 N  (maybe)
                 ]

feeds_to_work = ['https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-nqrw',
                 'https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs',
                 'https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs'
                 ]

train_names_to_work = ['Q',
                       '2 3',
                       '1',
                       ]

stop_names_long = ['72nd street, Q South',
                   '42nd street, 2 or 3 North',
                   '96th street, 1 North']

travel_time_to_next = [8,
                       7,
                       4
                       ]
"""



# my_stop_id = '127N' 
# my_stop_id = '120N' # 96 123 N  (maybe)
# my_stop_id = '120S' # 96 123 S (maybe)
# my_stop_id = 'XX' # 


# How long does it take in minutes to walk to this station?
my_walking_time = 0

# How long should we wait before refreshing MTA feed? MTA says feeds are
# generated every 30 seconds, so anything less wouldn't make sense.
# Feed size (from each pull) can be around 500 kiB. Therefore expect to pull
# around 350 MiB in a 24 hour period.
# On a RaspPi Zero, expect a turn-around of retrieving and processing data
# to be around 15 seconds.
feed_refresh_delay = 65

def get_trains(action, current_feed_url, stop_index):
    global feed
    
    # Determine if we should do a new pull or a refresh according to NYCT-GTFS
    # library instruction. We must also update feed references as existing
    # objects are not modified by refresh()
    if action == 'refresh':
        feed.refresh()
        trains = feed.trips[stop_index]
    else:
        feed = NYCTFeed(current_feed_url, api_key=my_api_key)


    #print(f"feed trips: {feed.trips}")

    # filter feed to only trains headed to our stop and are moving
    trains = feed.filter_trips(headed_for_stop_id=stops_to_work[stop_index], underway=True)


    #print("trains variable is" + str(trains)) #TW did this
    # find trains those next stop is our stop
    # (i.e. we don't want those earlier down the track)
    arrivals = []
    for train_num in range(len(trains)):
        stops = trains[train_num].stop_time_updates
        for stop in stops:
            if stop.stop_id == stops_to_work[stop_index]:

                # get arrival time (in minutes) and train number.
                # Ignore trains arriving too soon
                arrival_time = int((stop.arrival - datetime.datetime.now())
                    .total_seconds() / 60)
                if arrival_time >= my_walking_time:
                  #  arrivals.append([arrival_time, trains[train_num].route_id])

                    arrivals.append([arrival_time, trains[train_num].route_id, stop.arrival, travel_time_to_next[stop_index]])

    arrivals.sort()
    #print("arrivals variable (in get_trains function), after sort, is: " + str(arrivals))
    return arrivals

def print_trains(train_line_name, train_list):
  """
  For a train arrival time list with multiple lines, prints only the trains of the line past
  TW function
  """
  for i in train_list:
    if i[1] in train_line_name and i[0] <= MAX_WAIT_TO_PRINT:
      #time_to_arrive = datetime.time(minute=i[3])
      arrive_time = i[2] + datetime.timedelta(0, 60*i[3])
      #print(f": {i}")
      print(f"{i[1]} train in {i[0]}' {i[2].strftime('%I:%M:%S')} --> {arrive_time.strftime('%I:%M:%S')}")


def main(stop_index):

    # get train data
    trains = get_trains('new', feeds_to_work[stop_index], stop_index)
                        
                       # my_feed_url)

    # TW VERSION: prints out arrival times in "trains" variable, only if second elemnt of each 
    # element is 'Q' then waits (feed_fresh_day) and refreshes trains variable and does it again
    #while True:
    print_trains(train_names_to_work[stop_index], trains)
    #time.sleep(feed_refresh_delay)
    #trains = get_trains('refresh')

   # print('All the trains: ' + str(trains))

now = datetime.datetime.now()

#print(now.strftime('%Y/%m/%d %H:%M:%S')) #24-hour format.
#print(now.strftime('%Y/%m/%d %I:%M:%S')) #12-hour format.


#date_now = datetime.datetime.now().time()
print(f"Checked at: {now.strftime('%I:%M:%S')}")

### MAIN ACTION HERE
        
for i in range(len(stops_to_work)):
  #date_now = datetime.datetime.now().strftime("%Y-%m-%d %H-%M-%S")
  #date_now = datetime.datetime.now().time()
  #print(f"date_now is {date_now}")
  print(f"\n** {stop_names_long[i]} **\n")
  # print(f"main loop, trying index: {i}")
  main(i)

#main(0)


Checked at: 12:17:29

** 116 street, 1 South **

1 train in 11' 12:29:15 --> 12:33:15

** 96 Street, 2 or 3 South **

2 train in 7' 12:25:20 --> 12:33:20
2 train in 25' 12:42:59 --> 12:50:59

** 42nd street, Q North **

Q train in 18' 12:36:27 --> 12:44:27
Q train in 34' 12:52:27 --> 01:00:27
Q train in 43' 01:01:30 --> 01:09:30
